## Exercise 2 - Naive Bayes

First name: Brian
<br>
Last name: Schweigler
<br>
Matriculation number: 16-102-071

#### (1a) In the correlation visualization, select the two features that have the most significant correlation to the target feature, Survived.

First some imports and preprocessing

In [5]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.feature_selection import chi2
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn import preprocessing
import scipy
from mlxtend.classifier import OneRClassifier
from mlxtend.evaluate import accuracy_score

df = pd.read_csv("data/titanic.csv", index_col='Name')
# df.describe(include='all')
pd.set_option('display.max_colwidth', None)
print(df.head(5))
le = preprocessing.LabelEncoder()
#df["Name"] = le.fit_transform(df["Name"])
df["Survived"] = le.fit_transform(df["Survived"])
df["Sex"] = le.fit_transform(df["Sex"])

                                                    Survived  Pclass     Sex  \
Name                                                                           
Mr. Owen Harris Braund                                     0       3    male   
Mrs. John Bradley (Florence Briggs Thayer) Cumings         1       1  female   
Miss. Laina Heikkinen                                      1       3  female   
Mrs. Jacques Heath (Lily May Peel) Futrelle                1       1  female   
Mr. William Henry Allen                                    0       3    male   

                                                     Age  \
Name                                                       
Mr. Owen Harris Braund                              22.0   
Mrs. John Bradley (Florence Briggs Thayer) Cumings  38.0   
Miss. Laina Heikkinen                               26.0   
Mrs. Jacques Heath (Lily May Peel) Futrelle         35.0   
Mr. William Henry Allen                             35.0   

                  

In [6]:
correlation = df.corr()["Survived"]
correlation = correlation.apply(lambda entry: abs(entry))

print("Correlation of columns to column survived")
print(correlation.sort_values(ascending=False))

correlation.pop(correlation.idxmax())
temp_correlation = correlation
best = correlation.idxmax()
temp_correlation.pop(best)
second_best = temp_correlation.idxmax()

print(best + " has the highest correlation, " + second_best + " the second highest")

Correlation of columns to column survived
Survived                   1.000000
Sex                        0.542152
Pclass                     0.336528
Fare                       0.256179
Parents/Children Aboard    0.080097
Age                        0.059665
Siblings/Spouses Aboard    0.037082
Name: Survived, dtype: float64
Sex has the highest correlation, Pclass the second highest


#### (1b) Using Naive Bayes classifier and the most two significant features to predict the Survival of the travellers.

In [11]:
naive_bayes = GaussianNB()
x_train, x_test, y_train, y_test = train_test_split(df[[best, second_best]], df["Survived"], test_size=0.4,
                                                    random_state=6)
naive_bayes.fit(x_train, y_train)
test_predictions = naive_bayes.predict(x_test)
accuracy = accuracy_score(y_test, test_predictions)
print("Accuracy is for Naive Bayes ", round(accuracy, 3))

Accuracy is for Naive Bayes  0.803


#### (1c) Compare the performance of your model when using all the attributes of the travellers.

In [13]:
naive_bayes_all = GaussianNB()
x_train_all, x_test_all, y_train_all, y_test_all = train_test_split(df.loc[:, df.columns != 'Survived'], df["Survived"],
                                                                    test_size=0.4, random_state=6)
naive_bayes_all.fit(x_train_all, y_train_all)
test_predictions_all = naive_bayes_all.predict(x_test_all)
accuracy_all = accuracy_score(y_test_all, test_predictions_all)
print("Accuracy using all attributes for Naive Bayes ", round(accuracy_all, 3))

Accuracy using all attributes for Naive Bayes  0.752


Thus using all columns is not only less efficient, but also performs worse.


#### (2a) Select the two features that have the most signifcant correlation to the target feature, daily return.

First simply import the dataset and set up the values:

In [18]:
pd.set_option('display.max_colwidth', None)
stock_df = pd.read_csv("data/Nasdaq.csv", index_col='Date')
print(stock_df.head(5))
# stock_df['Date'] = stock_df['Date'].apply(pd.to_datetime)
stock_df.describe(include='all')


daily_return = np.empty(stock_df['Close'].shape)
#  From Slides: Daily return (r): Difference in percentage between the price at time t+1 and at time t
daily_return[0] = float('NaN') # The first
daily_return[1:] = np.ediff1d(stock_df['Close']) / stock_df['Close'][:-1]
stock_df.insert(loc=len(stock_df.columns), column='Daily Return', value=daily_return)

# for i in range(1, len(stock_df)):
#     p_t = stock_df.loc[i, 'Close']
#     p_t_minus_1 = stock_df.loc[i - 1, 'Close']
#     daily_return = (p_t - p_t_minus_1) / p_t_minus_1
#     stock_df.loc[i, 'Daily Return'] = daily_return

stock_df['Rolling Mean 5'] = stock_df['Close'].rolling(5).mean()
stock_df['Rolling Mean 10'] = stock_df['Close'].rolling(10).mean()
stock_df['Rolling Mean 20'] = stock_df['Close'].rolling(20).mean()
stock_df['Rolling Mean 50'] = stock_df['Close'].rolling(50).mean()
stock_df['Rolling Mean 200'] = stock_df['Close'].rolling(200).mean()
stock_df = stock_df.fillna(0)  # NAs replaced with zero

print(stock_df.tail(5))

                  Open        High         Low       Close   Adj Close  Volume
Date                                                                          
1971-02-05  100.000000  100.000000  100.000000  100.000000  100.000000       0
1971-02-08  100.839996  100.839996  100.839996  100.839996  100.839996       0
1971-02-09  100.760002  100.760002  100.760002  100.760002  100.760002       0
1971-02-10  100.690002  100.690002  100.690002  100.690002  100.690002       0
1971-02-11  101.449997  101.449997  101.449997  101.449997  101.449997       0
                    Open          High           Low         Close  \
Date                                                                 
2021-09-15  15071.339844  15174.379883  14984.679688  15161.530273   
2021-09-16  15120.089844  15205.500000  15047.139648  15181.919922   
2021-09-17  15163.360352  15166.559570  14998.730469  15043.969727   
2021-09-20  14758.139648  14841.820312  14530.070312  14713.900391   
2021-09-21  14803.400391  1

Getting started with the task:
